In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from convolution_engine.engine import ConvolutionEngine
from convolution_engine.padding_engine.padding_engine import PaddingEngine

# Check available backends
print("Convolution backends:", ConvolutionEngine.list_available_backends())
print("Padding backends:", PaddingEngine.list_available_backends())

# Convolution Engine Examples

This notebook demonstrates the convolution engine's core functionality:

1. **Basic 1D Convolution** - Single vector smoothing
2. **Backend Selection** - Performance-oriented backend choices
3. **Batched Processing** - Multiple vectors at once (2D arrays)
4. **Padding Modes** - Boundary handling before convolution
5. **Complete Workflow** - Pad → Convolve → Extract pattern

In [ ]:
# Create simple data and Gaussian kernel
data = np.array([1, 2, 5, 8, 5, 2, 1], dtype=np.float32)
kernel = np.array([0.25, 0.5, 0.25])  # 3-element smoothing kernel

# Initialize engine and convolve
engine = ConvolutionEngine(backend='numpy')
result = engine.convolve(data, kernel)

print(f"Input shape:  {data.shape}")
print(f"Kernel shape: {kernel.shape}")
print(f"Output shape: {result.shape}")
print(f"\nInput:  {data}")
print(f"Output: {result}")
print(f"\nNote: Output is {len(kernel)-1} elements shorter (valid mode, no padding)")

In [ ]:
# Demonstrate different padding modes
data = np.array([10, 20, 30, 40, 50], dtype=np.float32)
pad_engine = PaddingEngine(backend='numpy_native')
pad_width = 3

modes = ['reflect', 'symmetric', 'edge', 'constant', 'wrap']

print(f"Original data: {data}\n")
print(f"{'Mode':<12} {'Padded Result'}")
print("-" * 50)

for mode in modes:
    if mode == 'constant':
        padded = pad_engine.pad(data, pad_width, mode=mode, constant_value=0)
    else:
        padded = pad_engine.pad(data, pad_width, mode=mode)
    print(f"{mode:<12} {padded}")

In [ ]:
# Realistic workflow: smoothing signal with boundary handling
signal = np.sin(np.linspace(0, 4*np.pi, 100))
kernel = np.ones(11) / 11  # 11-element moving average

# Step 1: Pad signal to preserve boundaries
pad_engine = PaddingEngine(backend='numpy_native')
pad_width = len(kernel) // 2  # Pad enough to get same-size output
padded_signal = pad_engine.pad(signal, pad_width, mode='reflect')

# Step 2: Convolve
conv_engine = ConvolutionEngine(backend='numpy')
convolved = conv_engine.convolve(padded_signal, kernel)

# Step 3: Verify output size matches input
print(f"Original: {signal.shape} → Padded: {padded_signal.shape} → "
      f"Convolved: {convolved.shape}")
assert convolved.shape == signal.shape, "Output size mismatch!"

# Visualize
plt.figure(figsize=(10, 4))
plt.plot(signal, 'o-', label='Original', alpha=0.5)
plt.plot(convolved, 's-', label='Smoothed', alpha=0.8)
plt.legend()
plt.title('Signal Smoothing with Boundary Preservation')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Process multiple signals efficiently
batch_signals = np.random.randn(20, 200).astype(np.float32)
kernel = np.ones(15) / 15

# Pad entire batch at once
pad_engine = PaddingEngine(backend='numba_numpy')  # Fast for batches
pad_width = len(kernel) // 2
padded_batch = pad_engine.pad(batch_signals, pad_width, mode='edge')

# Convolve entire batch at once
conv_engine = ConvolutionEngine(backend='numba')  # Fast for batches
smoothed_batch = conv_engine.convolve(padded_batch, kernel)

print(f"Batch shape: {batch_signals.shape}")
print(f"Padded: {padded_batch.shape}")
print(f"Smoothed: {smoothed_batch.shape}")
print(f"\nProcessed {batch_signals.shape[0]} signals efficiently in one operation")

In [ ]:
# Demonstrate arbitrary dimensional support across all padding modes
import torch

print("="*70)
print("PYTORCH BACKENDS: ARBITRARY DIMENSIONS × ALL PADDING MODES")
print("="*70)

# Test different dimensionalities
test_cases = [
    (torch.randn(200), "1D"),
    (torch.randn(10, 200), "2D"),
    (torch.randn(8, 5, 200), "3D"),
    (torch.randn(4, 8, 8, 200), "4D"),
]

engine = PaddingEngine(backend='pytorch_cpu')
conv_engine = ConvolutionEngine(backend='pytorch_cpu')
kernel = torch.ones(15) / 15
pad_width = len(kernel) // 2

modes = ['reflect', 'symmetric', 'edge', 'constant', 'wrap']

for data, dim_label in test_cases:
    print(f"\n{dim_label}: shape {tuple(data.shape)}")
    print(f"{'Mode':<12} {'Output Shape':<20} {'Preserved':<10}")
    print("-" * 42)

    for mode in modes:
        padded = engine.pad(data, pad_width, mode=mode)
        smoothed = conv_engine.convolve(padded, kernel)

        preserved = "✓" if smoothed.shape == data.shape else "✗"
        print(f"{mode:<12} {str(tuple(smoothed.shape)):<20} {preserved:<10}")

print(f"\n✓ All modes work with arbitrary dimensions")
print(f"✓ Shape preservation: input shape == output shape")

### Backend-Specific kwargs Reference

**This is an advanced feature that breaks portability. Use with caution.**

| Backend | Underlying Function | Available kwargs | Example |
|---------|-------------------|------------------|---------|
| `numpy` | `np.convolve()` | `mode`: 'valid', 'same', 'full' | `mode='same'` |
| `scipy` | `scipy.ndimage.correlate1d()` | `mode`: 'reflect', 'constant', 'nearest', 'mirror', 'wrap'<br>`cval`: constant value | `mode='reflect'` |
| `pytorch_cpu/gpu` | `F.conv1d()` | `padding`: int or tuple<br>`stride`: int<br>`dilation`: int | `padding=2` |
| `numba` | Custom implementation | None (not supported) | - |

**Recommendation:** Only use kwargs when:
1. You know you'll never switch backends
2. You need features unavailable in PaddingEngine
3. You document the backend dependency clearly

For portable code, always use: `PaddingEngine.pad()` → `ConvolutionEngine.convolve()`

In [ ]:
# WARNING: Using kwargs bypasses the engine's interface and accesses underlying backend APIs
# This is NOT guaranteed to work across backends and may break if you switch backends

data = np.array([1, 2, 3, 4, 5], dtype=np.float32)
kernel = np.array([0.25, 0.5, 0.25])

print("Standard 'valid' mode behavior:")
print(f"Input length: {len(data)}, Output length: {len(data) - len(kernel) + 1}\n")

# NumPy backend: passes kwargs to np.convolve
engine_numpy = ConvolutionEngine(backend='numpy')
result_same = engine_numpy.convolve(data, kernel, mode='same')  # Output same size as input
result_full = engine_numpy.convolve(data, kernel, mode='full')  # Output is len(data) + len(kernel) - 1

print(f"NumPy backend with mode='same': {result_same} (length {len(result_same)})")
print(f"NumPy backend with mode='full': {result_full} (length {len(result_full)})")

# SciPy backend: passes kwargs to correlate1d
engine_scipy = ConvolutionEngine(backend='scipy')
result_reflect = engine_scipy.convolve(data, kernel, mode='reflect')  # Reflect padding

print(f"SciPy backend with mode='reflect': {result_reflect} (length {len(result_reflect)})")

# PyTorch backend: passes kwargs to F.conv1d
if ConvolutionEngine.list_available_backends().get('pytorch_cpu'):
    engine_pytorch = ConvolutionEngine(backend='pytorch_cpu')
    # padding parameter for conv1d: int or tuple
    result_padded = engine_pytorch.convolve(data, kernel, padding=1)  # Pad with 1 zero on each side
    print(f"PyTorch backend with padding=1: {result_padded} (length {len(result_padded)})")

print("\n⚠️  CAUTION:")
print("- These kwargs are backend-specific and undocumented")
print("- Switching backends will likely break your code")
print("- Use PaddingEngine + standard convolve() for portable code")

In [ ]:
"""
Substrate Boundary Condition Padding

Pads substrate base with zero-temperature BC (heat sink).
"""

import numpy as np
from numpy.typing import NDArray


class SubstrateBoundaryPadding:
    """Zero-temperature BC at substrate base."""

    def __call__(
        self,
        temperature: NDArray[np.float64],
        substrate_thickness: int,
        pad_width: int,
        axis: int
    ) -> NDArray[np.float64]:
        """Pad with substrate boundary conditions.

        Args:
            temperature: Temperature field (nx, ny, nz)
            substrate_thickness: Substrate thickness in voxels along axis
            pad_width: Padding width in voxels
            axis: Substrate axis (0=x, 1=y, 2=z)

        Returns:
            Padded temperature with BCs applied
        """
        if axis not in (0, 1, 2):
            raise ValueError(f"axis must be 0, 1, or 2, got {axis}")

        # Single pad: bottom of axis, both sides of other axes
        pad_config = [(pad_width, pad_width)] * 3
        pad_config[axis] = (pad_width, 0)  # Bottom only for substrate axis

        T_padded = np.pad(temperature, pad_config, mode='reflect')

        # Negate substrate regions
        # Bottom (entire bottom padding)
        slices = [slice(None)] * 3
        slices[axis] = slice(0, pad_width)
        T_padded[tuple(slices)] *= -1

        # Sides (only substrate thickness region)
        for side_axis in range(3):
            if side_axis == axis:
                continue

            # Left side substrate
            slices = [slice(None)] * 3
            slices[side_axis] = slice(0, pad_width)
            slices[axis] = slice(pad_width, pad_width + substrate_thickness)
            T_padded[tuple(slices)] *= -1

            # Right side substrate
            slices = [slice(None)] * 3
            slices[side_axis] = slice(-pad_width, None)
            slices[axis] = slice(pad_width, pad_width + substrate_thickness)
            T_padded[tuple(slices)] *= -1

        return T_padded

In [ ]:
# Create test field: substrate at bottom
T = np.ones((10, 10, 20)) * 500
substrate_thickness = 5


padding = SubstrateBoundaryPadding()
T_padded = padding(T, substrate_thickness=5, pad_width=2, axis=2)

In [ ]:
T_padded

## Masked Convolution Engine

Handles gappy signals by grouping valid regions by length and batch processing each group.

**Key features:**
- Finds valid regions in masked data
- Groups regions by length → minimal batch operations
- Each batch processed in parallel via Numba/PyTorch
- User controls padding strategy via `pad_func`

In [ ]:
from convolution_engine.masked_convolution_engine import MaskedConvolutionEngine
from convolution_engine.padding_engine.padding_engine import PaddingEngine

# Setup engines
engine = MaskedConvolutionEngine.create_fast()  # Uses Numba
padding_engine = PaddingEngine(backend='numba_numpy')
pad_func = lambda x, pw: padding_engine.pad(x, pw, mode='reflect')

# Signal with gaps (NaN = invalid/masked)
signal = np.array([1, 2, 3, np.nan, np.nan, 6, 7, 8, 9, np.nan, 11, 12, 13], dtype=float)
mask = np.isfinite(signal)
kernel = np.array([0.25, 0.5, 0.25])  # Smoothing kernel

# Convolve with batch processing
result = engine.convolve(
    signal=signal,
    kernel=kernel,
    mask=mask,
    pad_width=len(kernel)//2,
    pad_func=pad_func
)

# Visualize
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Input with gaps
x = np.arange(len(signal))
valid = ~np.isnan(signal)
ax1.plot(x[valid], signal[valid], 'o-', label='Valid data', markersize=6)
ax1.plot(x[~valid], [0]*np.sum(~valid), 'rx', label='Gaps', markersize=10)
ax1.set_ylabel('Input')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Output with gaps preserved
valid_out = ~np.isnan(result)
ax2.plot(x[valid_out], result[valid_out], 's-', label='Smoothed', markersize=6, color='C1')
ax2.plot(x[~valid_out], [0]*np.sum(~valid_out), 'rx', label='Gaps preserved', markersize=10)
ax2.set_xlabel('Index')
ax2.set_ylabel('Output')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Masked Convolution: Batch-by-Length Processing')
plt.tight_layout()
plt.show()

print(f"\nEngine info:")
print(f"  Region finding: {engine.region_engine.backend}")
print(f"  Convolution: {engine.convolution_engine.backend_name}")